In [1]:
import numpy as np
import matplotlib.pyplot as plt

In [2]:
X = np.array([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
              [1, 1, 2, 1, 3, 0, 5, 10, 1, 2],
              [500, 700, 750, 600, 1450,
               800, 1500, 2000, 450, 1000],
              [1, 1, 2, 1, 2, 
               1, 3, 3, 1, 2]], dtype = np.float64)
y = np.array([0, 0, 1, 0, 1, 0, 1, 0, 1, 1], dtype = np.float64)

In [3]:
def calc_std_feat(x):
  res = (x - x.mean()) / x.std()
  return res

In [4]:
X_st = X.copy()
X_st[2, :] = calc_std_feat(X[2, :])

In [5]:
def calc_logloss(y, y_pred):
  err = - np.mean(y * np.log(y_pred) + (1.0 - y) * np.log(1.0 - y_pred))
  err = np.sum(err)
  return err

In [6]:
def sigmoid(z):
  res = 1 / (1 + np.exp(-z))
  return res

In [7]:
### Logistic Regression

In [8]:
def eval_model(X, y, iterations, alpha=1e-4):
  np.random.seed(42)
  W = np.random.randn(X.shape[0])
  n = X.shape[1]
  for i in range(1, iterations+1):
    z = np.dot(W, X)
    y_pred = sigmoid(z)
    err = calc_logloss(y, y_pred)
    W -= alpha * (1/n * np.dot((y_pred - y), X.T))
    if i % (iterations / 10) == 0:
      print(i, W, err)
  return W

**1*. Измените функцию calc_logloss так, чтобы нули по возможности не попадали в np.log.**


In [9]:
### Чтобы нули по возможности не попадали в np.log, y_pred не должен равнятся 0 или 1

In [10]:
def calc_logloss_nplog_without_zero(y, y_pred):
    y_pred_not_nplogzero = y_pred.copy()
    for i in range(len(y_pred_not_nplogzero)):
        if y_pred_not_nplogzero[i] == 0.0:
            y_pred_not_nplogzero[i] = y_pred_not_nplogzero + 1e-100
        elif y_pred_not_nplogzero[i] == 1.0:
            y_pred_not_nplogzero[i] = y_pred_not_nplogzero + 1e-100
#     print(y_pred_not_nplogzero + 1e-100)
    err = - np.mean(y * np.log(y_pred_not_nplogzero) + (1.0 - y) * np.log(1.0 - y_pred_not_nplogzero))
    err = np.sum(err)
    return err

In [11]:
# Пример применения
y1 = np.array([1, 0])
y_pred1 = np.array([0.9, 0.1])
calc_logloss(y1, y_pred1)
calc_logloss_nplog_without_zero(y1, y_pred1)

0.10536051565782628

**2. Подберите аргументы функции eval_model для логистической регрессии таким образом, чтобы log loss был минимальным.**


In [12]:
W = eval_model(X_st, y, iterations=1000, alpha=3)

100 [-8.51673763 -1.71111931 -1.38606982  7.28158728] 0.2931793344885868
200 [-13.11264873  -2.91758817  -1.32887419  15.25013145] 1.7793351273150546
300 [-16.34784118  -2.52357282  -3.58710026  14.06490789] 0.23209075103450272
400 [-17.91804846  -2.48236469  -4.2159153   15.047786  ] 0.2080492405336928
500 [-19.2236722   -2.56735521  -4.65327832  16.09142577] 0.19997871379404897
600 [-20.4198482   -2.5623619   -5.04609261  17.01262201] 0.18797980004953924
700 [-21.51452046  -2.58394494  -5.39959367  17.86189333] 0.17921032579950585
800 [-22.52683467  -2.60985149  -5.7208322   18.65211712] 0.17189744162163184
900 [-23.4700611   -2.6373771   -6.01607687  19.39263502] 0.165642859305829
1000 [-24.35402284  -2.66399475  -6.28979027  20.09126347] 0.16018416662246715


**3. Создайте функцию calc_pred_proba, возвращающую предсказанную вероятность класса 1 (на вход подаются W, который уже посчитан функцией eval_model и X, на выходе - массив y_pred_proba).**


In [13]:
def calc_pred_proba(W, X):
    return sigmoid(np.dot(X.T, W))

In [14]:
calc_pred_proba(W, X_st)

array([0.31748793, 0.03358156, 0.99999851, 0.11279739, 0.84225541,
       0.11996466, 0.99999986, 0.01770817, 0.47084048, 0.99996194])

**4. Создайте функцию calc_pred, возвращающую предсказанный класс (на вход подаются W, который уже посчитан функцией eval_model и X, на выходе - массив y_pred).**


In [15]:
def calc_pred(W, X, t):
    y_pred_bool = sigmoid(np.dot(W,X)) > t
    y_pred = y_pred_bool.astype(int)
    return y_pred

In [16]:
calc_pred(W, X_st, 0.5)

array([0, 0, 1, 0, 1, 0, 1, 0, 0, 1])

**5. Посчитайте Accuracy, матрицу ошибок, точность и полноту, а также F1 score.**


In [17]:
def metrics(y, y_pred):
    TP = np.sum(np.logical_and(y_pred == 1, y == 1))
    TN = np.sum(np.logical_and(y_pred == 0, y == 0))
    FP = np.sum(np.logical_and(y_pred == 1, y == 0))
    FN = np.sum(np.logical_and(y_pred == 0, y == 1))

    print(f'TP:{TP}\tTN:{TN}\tFP:{FP}\tFN:{FN}')
    
    accuracy = (TP + TN)/(TP + TN + FP + FN)
    precision = TP/(TP + FP)
    recall = TP/(TP + FN)
    f1_score = 2*precision*recall/(precision + recall)

    return accuracy, precision, recall, f1_score

In [18]:
y_pred = calc_pred(W, X_st, 0.5)

In [19]:
metrics(y, y_pred)

TP:4	TN:5	FP:0	FN:1


(0.9, 1.0, 0.8, 0.888888888888889)

**6. Могла ли модель переобучиться? Почему?**


Модель могла переобучиться так как мало данных.

**7*. Создайте функции eval_model_l1 и eval_model_l2 с применением L1 и L2 регуляризаций соответственно.**

In [20]:

def eval_model_l1(X, y, iterations, alpha=1e-4, lambda_=1e-8):
    np.random.seed(42)
    W = np.random.randn(X.shape[0])
    n = X.shape[1]
    for i in range(1, iterations+1):
        z = np.dot(W, X)
        y_pred = sigmoid(z)
        err = calc_logloss(y, y_pred)
        W -= alpha * (1/n * np.dot((y_pred - y), X.T) + lambda_ * np.sign(W))
        if i % (iterations / 10) == 0:
            print(i, W, err)
    return W

In [21]:
def eval_model_l2(X, y, iterations, alpha=1e-4, lambda_=1e-8):
    np.random.seed(42)
    W = np.random.randn(X.shape[0])
    n = X.shape[1]
    for i in range(1, iterations+1):
        z = np.dot(W, X)
        y_pred = sigmoid(z)
        err = calc_logloss(y, y_pred)
        W -= alpha * (1/n * 2 * np.dot((y_pred - y), X.T) + lambda_ * W) 
        if i % (iterations / 10) == 0:
            print(i, W, err)
    return W

In [22]:
def eval_model(X, y, iterations, alpha=1e-4):
  np.random.seed(42)
  W = np.random.randn(X.shape[0])
  n = X.shape[1]
  for i in range(1, iterations+1):
    z = np.dot(W, X)
    y_pred = sigmoid(z)
    err = calc_logloss(y, y_pred)
    W -= alpha * (1/n * np.dot((y_pred - y), X.T))
    if i % (iterations / 10) == 0:
      print(i, W, err)
  return W

In [23]:
# L2
W_l2 = eval_model_l2(X_st, y, 10**4, alpha=1e-4)
metrics(y, calc_pred(W_l2, X_st, t=0.5))

1000 [ 0.42368488 -0.36360644  0.6425442   1.41506564] 0.8808932704629179
2000 [ 0.36455302 -0.53308995  0.64183938  1.33336551] 0.6825763394547726
3000 [ 0.32087991 -0.62089144  0.65260746  1.28542859] 0.6200814713666538
4000 [ 0.28678646 -0.66213495  0.66947194  1.25797133] 0.6000644467180752
5000 [ 0.25808522 -0.68190636  0.68834815  1.24118928] 0.590676253168305
6000 [ 0.23270311 -0.69102792  0.70761907  1.23041869] 0.5845657025663409
7000 [ 0.20953961 -0.69460916  0.7265977   1.22327246] 0.5797505709386593
8000 [ 0.18797086 -0.69529889  0.74496785  1.21844005] 0.5756135053758216
9000 [ 0.16762795 -0.69453745  0.76258436  1.21517407] 0.5719342307075242
10000 [ 0.14828476 -0.6931214   0.77938921  1.21303834] 0.5686172447837053
TP:4	TN:2	FP:3	FN:1


(0.6, 0.5714285714285714, 0.8, 0.6666666666666666)

In [24]:
# L1
W_l1 = eval_model_l1(X_st, y, 10**4, alpha=1e-4)
metrics(y, calc_pred(W_l1, X_st, t=0.5))

1000 [ 0.45886878 -0.25439917  0.64531344  1.46695851] 1.0395158246720007
2000 [ 0.4236874  -0.36359967  0.64254389  1.41506965] 0.8807599511039926
3000 [ 0.39201058 -0.45900098  0.64080604  1.3697477 ] 0.7587599809788
4000 [ 0.36455824 -0.53307003  0.64184068  1.33337513] 0.6825375011952257
5000 [ 0.34113652 -0.5852622   0.64604925  1.30592473] 0.6416846622035376
6000 [ 0.32088548 -0.62087069  0.65260867  1.28543874] 0.6200744626141542
7000 [ 0.30297172 -0.64525902  0.66061929  1.26993179] 0.6077787492004819
8000 [ 0.28679128 -0.66211921  0.66947216  1.25797949] 0.6000619425059602
9000 [ 0.27192765 -0.67381776  0.6787924   1.24862025] 0.5947176150979027
10000 [ 0.25808923 -0.68189546  0.68834748  1.24119539] 0.5906743813144024
TP:5	TN:1	FP:4	FN:0


(0.6, 0.5555555555555556, 1.0, 0.7142857142857143)